In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/final/final.csv")

In [4]:
df.season.unique()

array([2017, 2016])

In [6]:
df.isna().sum()

player                0
age_x                 0
overall rating        0
potential            10
team & contract       0
                     ..
Performance_Int      80
Performance_TklW     80
Performance_PKwon    80
Performance_PKcon    80
Performance_OG       80
Length: 200, dtype: int64

In [147]:
# Fixed TRAINING_FEATURES - remove duplicates with FEATURES_EXCLUDE
TRAINING_FEATURES = [
    # Demographics & physical - REMOVED skill_moves, international_reputation (they're excluded)
    
    # Playing time
    "Playing_Time_Min",
    "Playing_Time_90s",
    "Starts_Starts",

    # Attacking output (per 90)
    "Per_90_Minutes_Gls",
    "Per_90_Minutes_Ast",
    "Per_90_Minutes_G+A",
    "Per_90_Minutes_xG",
    "Per_90_Minutes_xAG",
    "Per_90_Minutes_npxG",

    # Shooting efficiency
    "Standard_SoT%",
    "Standard_G/Sh",
    "Expected_npxG/Sh",
    "Expected_G-xG",

    # Passing & creativity
    "KP",
    "Ast",
    "Total_PrgDist",
    "Progression_PrgP",
    "SCA_SCA90",
    "GCA_GCA90",

    # Carrying
    "Carries_PrgDist",
    "Carries_1/3",
    "Take-Ons_Succ",

    # Defensive contribution
    "Tkl+Int",
    "Int",
    "Blocks_Blocks",
    "Aerial_Duels_Won",

    # Discipline / reliability
    "Performance_CrdY",
    "Performance_CrdR",
    "Err",

    # Team context
    "Team_Success_PPM",
    "Team_Success_+/-90",
    "Team_Success_(xG)_xG+/-90"
]



TARGETS = [
    # Scoring threat (finishing quality & volume)
    "Per_90_Minutes_npxG",
    "Per_90_Minutes_xG",
    "Standard_Sh/90",

    # Creativity & chance creation
    "Per_90_Minutes_xAG",
    "KP",
    "SCA_SCA90",

    # Ball progression & buildup value
    "Progression_PrgP",
    "Progression_PrgC",
    "Carries_PrgDist",

    # Defensive contribution
    "Tkl+Int",
    "Blocks_Blocks",
    "Aerial_Duels_Won",

    # Involvement / usage
    "Touches_Touches",
    "Receiving_Rec",
]

# Keep FEATURES_EXCLUDE as is
FEATURES_EXCLUDE = [
    'player', 'season', 'team', 'nation', 'born', 'league', 'age',
    'height(cm)', 'weight(kg)', 'foot', 'general_position', 'best_position'
]

In [148]:
# Fix: Aggregate data per player-season (combine stats from multiple teams)
def aggregate_player_season(df):
    """Aggregate stats for players who played for multiple teams in same season"""
    
    # Identify numeric columns to sum/average
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    # Remove identifier columns
    numeric_cols = [c for c in numeric_cols if c not in ['season']]
    
    # Categorical columns to take the last value (most recent team)
    cat_cols = ['team', 'league', 'nation', 'born', 'foot', 'general_position', 'best_position']
    
    agg_dict = {}
    
    # Sum numeric stats
    for col in numeric_cols:
        agg_dict[col] = 'sum'
    
    # Take last value for categorical
    for col in cat_cols:
        if col in df.columns:
            agg_dict[col] = 'last'
    
    # Group by player-season and aggregate
    df_agg = df.groupby(['player', 'season']).agg(agg_dict).reset_index()
    
    return df_agg

df_clean = aggregate_player_season(df)

def make_samples(df, window, features_exclude, target_cols):
    """
    Create samples with lagged features for time series prediction.
    Each player gets exactly ONE row per target season.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe with player-season data
    window : int
        Number of historical seasons to use as features
    features_exclude : list
        Columns to exclude from lagging
    target_cols : list
        Target variables to predict
    
    Returns:
    --------
    pd.DataFrame
        Dataset with lagged features and targets
    """
    
    df = df.copy()
    df.sort_values(['player', 'season'], inplace=True)

    rows = []
    
    for player, g in df.groupby('player'):
        g = g.reset_index(drop=True)
        if len(g) <= window:
            continue
            
        # For each possible target season
        for i in range(window, len(g)):
            past = g.loc[i-window:i-1].copy()  # Historical seasons
            target_season = g.loc[i].copy()  # Target season
            
            flat = {}
            
            # Create lagged features
            for j, (_, r) in enumerate(past.iterrows(), start=1):
                suffix = f"_lag{window-j+1}"
                for c in TRAINING_FEATURES:  # Only lag training features
                    if c in features_exclude or c not in r.index:
                        continue
                    flat[c + suffix] = r[c]
            
            # Add static features from most recent season (no lag)
            flat['height(cm)'] = g.loc[i-1, 'height(cm)']
            flat['weight(kg)'] = g.loc[i-1, 'weight(kg)']
            flat['foot'] = g.loc[i-1, 'foot']
            flat['general_position'] = g.loc[i-1, 'general_position']
            flat['best_position'] = g.loc[i-1, 'best_position']
            
            # Add metadata
            flat['player'] = player
            flat['season_target'] = target_season['season']
            flat['age_at_target'] = g.loc[i-1, 'age'] + 1
            flat['team_last'] = g.loc[i-1, 'team']
            flat["league_last"] = g.loc[i-1, 'league']

            # Add targets
            for tcol in target_cols:
                if tcol in target_season.index:
                    flat[tcol] = target_season[tcol]
            
            rows.append(flat)
    
    return pd.DataFrame(rows)



# Create samples
df_samples = make_samples(
    df=df_clean,
    window=3,
    features_exclude=FEATURES_EXCLUDE,
    target_cols=TARGETS
)

In [149]:
lagged_cols = [c for c in df_samples.columns if 'lag' in c]
target_cols = [c for c in df_samples.columns if c in TARGETS]
static_cols = [c for c in df_samples.columns if c not in lagged_cols + target_cols and c not in ['player']]

In [150]:
df_samples = df_samples[['player'] +static_cols + lagged_cols + target_cols]

# Training

In [151]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.multioutput import MultiOutputRegressor
import pandas as pd

In [152]:
feature_cols = lagged_cols + [
    'height(cm)', 'weight(kg)', 'foot', 'general_position', 'best_position',
    'age_at_target', 'team_last', 'league_last'
]


df_clean_train = df_samples.dropna(subset=TARGETS).copy()
print(f"Samples after removing missing targets: {len(df_clean_train)}")


df = df_clean_train.copy()
for target in TARGETS:
    season_means = df.groupby('season_target')[target].mean()
    for season in season_means.index:
        mask = df['season_target'] == season
        df.loc[mask, target] = df.loc[mask, target] / season_means[season]

# Split the normalized data
train_mask = df['season_target'] < 2024
test_mask = df['season_target'] >= 2024

X_train_norm = X[train_mask]
y_train_norm = df.loc[train_mask, TARGETS]
X_test_norm = X[test_mask] 
y_test_norm = df.loc[test_mask, TARGETS]

Samples after removing missing targets: 2562


In [153]:
# Handle categorical vairables

categorical_features = ['foot', 'general_position', 'best_position', 'team_last', 'league_last']
le_dict = {}

X = df[feature_cols].copy()
y = df[TARGETS].copy()

# Encode categorical variables
for col in categorical_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    le_dict[col] = le

In [154]:
# 3. Train/test split by season
print("\n📊 Splitting data by season...")
train_mask = df['season_target'] < 2024
test_mask = df['season_target'] >= 2024

X_train = X[train_mask]
y_train = y[train_mask]
X_test = X[test_mask]
y_test = y[test_mask]

print(f"Training samples: {X_train.shape[0]} (seasons < 2024)")
print(f"Test samples: {X_test.shape[0]} (seasons >= 2024)")



📊 Splitting data by season...
Training samples: 1271 (seasons < 2024)
Test samples: 1291 (seasons >= 2024)


In [155]:
print("\n⚙️ Scaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


⚙️ Scaling features...


In [156]:
# 5. Train Multi-output model
model_simple = MultiOutputRegressor(RandomForestRegressor(
    n_estimators=50,      # Fewer trees
    max_depth=10,         # Limit depth
    min_samples_split=20, # More conservative splits
    min_samples_leaf=10,  # Larger leaf sizes
    random_state=42,
    n_jobs=-1
))

model_simple.fit(X_train_scaled, y_train)


,estimator estimator: estimator objectAn estimator object implementing :term:`fit` and :term:`predict`.,RandomForestR...ndom_state=42)
,"n_jobs n_jobs: int or None, optional (default=None)The number of jobs to run in parallel.:meth:`fit`, :meth:`predict` and :meth:`partial_fit` (if supportedby the passed estimator) will be parallelized for each target.When individual estimators are fast to train or predict,using ``n_jobs > 1`` can result in slower performance dueto the parallelism overhead.``None`` means `1` unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all available processes / threads.See :term:`Glossary ` for more details... versionchanged:: 0.20 `n_jobs` default changed from `1` to `None`.",None
,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",50
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",10
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",20
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",10
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least 

In [157]:
# 6. Evaluate model_simple
print("\n📈 Evaluating model_simple...")
y_pred_train = model_simple.predict(X_train_scaled)
y_pred_test = model_simple.predict(X_test_scaled)


results = []
for i, target in enumerate(TARGETS):
    train_mae = mean_absolute_error(y_train.iloc[:, i], y_pred_train[:, i])
    test_mae = mean_absolute_error(y_test.iloc[:, i], y_pred_test[:, i])
    train_r2 = r2_score(y_train.iloc[:, i], y_pred_train[:, i])
    test_r2 = r2_score(y_test.iloc[:, i], y_pred_test[:, i])
    
    results.append({
        'target': target,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_r2': train_r2,
        'test_r2': test_r2
    })


📈 Evaluating model_simple...


In [158]:
results_df = pd.DataFrame(results)
print("\n🎯 Model Performance:")
results_df


🎯 Model Performance:


,target,train_mae,test_mae,train_r2,test_r2
0,Per_90_Minutes_npxG,0.295604,0.554224,0.739269,0.293989
1,Per_90_Minutes_xG,0.302097,0.534991,0.756696,0.348013
2,Standard_Sh/90,0.248660,0.427239,0.685209,0.318566
3,Per_90_Minutes_xAG,0.326394,0.551061,0.681003,0.268768
4,KP,0.360198,0.534114,0.694741,0.415894
5,SCA_SCA90,0.207433,0.379728,0.696339,0.314606
6,Progression_PrgP,0.335546,0.516340,0.642003,0.288867
7,Progression_PrgC,0.364210,0.570665,0.697809,0.354677
8,Carries_PrgDist,0.335303,0.503936,0.637555,0.327087
9,Tkl+Int,0.327482,0.491657,0.643877,0.285603


In [159]:
# 1. Check for data issues
print("🔍 DIAGNOSING THE PROBLEM...")
print("="*50)

# Check target value ranges
print("Target value distributions:")
for target in TARGETS:
    train_vals = y_train[target]
    test_vals = y_test[target]
    print(f"\n{target}:")
    print(f"  Train: min={train_vals.min():.2f}, max={train_vals.max():.2f}, mean={train_vals.mean():.2f}")
    print(f"  Test:  min={test_vals.min():.2f}, max={test_vals.max():.2f}, mean={test_vals.mean():.2f}")

# Check for NaN/infinity in features
print(f"\nFeature issues:")
print(f"NaN in X_train: {X_train_scaled.shape[0] - np.isfinite(X_train_scaled).all(axis=1).sum()}")
print(f"NaN in X_test: {X_test_scaled.shape[0] - np.isfinite(X_test_scaled).all(axis=1).sum()}")

# Check prediction ranges
print(f"\nPrediction ranges:")
for i, target in enumerate(TARGETS):
    train_pred_range = f"{y_pred_train[:, i].min():.2f} to {y_pred_train[:, i].max():.2f}"
    test_pred_range = f"{y_pred_test[:, i].min():.2f} to {y_pred_test[:, i].max():.2f}"
    print(f"{target}: Train preds={train_pred_range}, Test preds={test_pred_range}")

# Check sample sizes by season
print(f"\nSample distribution:")
train_seasons = df_clean_train[train_mask]['season_target'].value_counts().sort_index()
test_seasons = df_clean_train[test_mask]['season_target'].value_counts().sort_index()
print("Train seasons:", train_seasons.to_dict())
print("Test seasons:", test_seasons.to_dict())

🔍 DIAGNOSING THE PROBLEM...
Target value distributions:

Per_90_Minutes_npxG:
  Train: min=0.00, max=11.14, mean=1.00
  Test:  min=0.00, max=20.31, mean=1.00

Per_90_Minutes_xG:
  Train: min=0.00, max=10.93, mean=1.00
  Test:  min=0.00, max=18.74, mean=1.00

Standard_Sh/90:
  Train: min=0.03, max=8.44, mean=1.00
  Test:  min=0.02, max=9.73, mean=1.00

Per_90_Minutes_xAG:
  Train: min=0.00, max=9.59, mean=1.00
  Test:  min=0.00, max=9.26, mean=1.00

KP:
  Train: min=0.00, max=8.15, mean=1.00
  Test:  min=0.00, max=6.42, mean=1.00

SCA_SCA90:
  Train: min=0.00, max=8.23, mean=1.00
  Test:  min=0.00, max=6.32, mean=1.00

Progression_PrgP:
  Train: min=0.00, max=6.80, mean=1.00
  Test:  min=0.00, max=9.44, mean=1.00

Progression_PrgC:
  Train: min=0.00, max=6.27, mean=1.00
  Test:  min=0.00, max=7.45, mean=1.00

Carries_PrgDist:
  Train: min=0.00, max=6.77, mean=1.00
  Test:  min=0.00, max=7.36, mean=1.00

Tkl+Int:
  Train: min=0.00, max=5.92, mean=1.00
  Test:  min=0.00, max=6.30, mean=1.